In [ ]:
import pandas as pd
import glob
from typing import Tuple
import torch
from torch import nn
from torch.utils.data import Dataset
import numpy as np
from math import log

In [ ]:
# Get a list of file paths for all CSV files in the data/ folder
csv_files = glob.glob('../data/*.csv')

# Create an empty list to store the DataFrames
dfs = []

# Read each CSV file into a DataFrame and append it to the list
for file in csv_files:
    df = pd.read_csv(file)
    df["user"] = file.split("/")[-1].split(".")[0] # Extract the user name from the file path
    dfs.append(df)

# Concatenate all the DataFrames in the list
df = pd.concat(dfs)
df = df[[
    "user",
    "card_id",
    "review_th",
    "rating",
    "delta_t",
]]

print(f"Data points: {len(df)}")
df.head(20)

In [ ]:
display(df.describe())
df.hist(bins=100, figsize=(20, 15))

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
print(f"Using {device} device")

class ReviewsDataset(Dataset):
    """
    A PyTorch Dataset that provides access to the reviews data.

    Features:
    - delta_t_is_null: whether the delta_t is null
    - delta_t: time since last review
    - past_reviews:
        - past_rating_is_null: whether the past rating is null
        - past_rating: the rating of the past review
        - past_delta_t_is_null: whether the past delta_t is null
        - past_delta_t: the time since the past review
    """

    def __init__(self, df: pd.DataFrame, num_past_reviews: int) -> None:
        self.df = df
        self.num_past_reviews = num_past_reviews

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor]:
        row = self.df.iloc[index]

        delta_t_is_null = int(row["delta_t"] == -1)
        delta_t = log(1 + row["delta_t"]) if not delta_t_is_null else 0
        rating = 0 if row["rating"] == 1 else 1

        past_reviews = []
        for i in range(1, self.num_past_reviews + 1):
            past_row = self.df.iloc[index - i]

            if past_row["user"] != row["user"] or past_row["card_id"] != row["card_id"]:
                past_reviews.append(
                    [
                        1,
                        0,
                        1,
                        0,
                    ]
                )
                continue

            past_rating_is_null = 0
            past_rating = 0 if past_row["rating"] == 1 else 1
            past_delta_t_is_null = int(past_row["delta_t"] == -1)
            past_delta_t = log(1 + past_row["delta_t"]) if not past_delta_t_is_null else 0

            past_reviews.append(
                [
                    past_rating_is_null,
                    past_rating,
                    past_delta_t_is_null,
                    past_delta_t,
                ]
            )

        return (
            torch.tensor([delta_t_is_null, delta_t] + sum(past_reviews, []), dtype=torch.float32, device=device),
            torch.tensor([rating], dtype=torch.float32, device=device),
        )

In [ ]:
# Hyperparameters
batch_size = 64
num_reviews = 2
epochs = 1
training_size = 0.8

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2 + 4 * num_reviews, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)


model = NeuralNetwork().to(device)
print(model)
print()

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
dataset = ReviewsDataset(df, num_reviews)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0 or batch == size - 1:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model):
    print("Testing")
    print("-------------------------------")
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Split between training and testing
train_size = int(training_size * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")